In [ ]:
#!/usr/bin/env python3
'''
Main script
'''

import os
from pymatgen.io.vasp.inputs import Poscar
from pymatgen.core import Structure

from CrySPY.interface import select_code
from CrySPY.job.ctrl_job import Ctrl_job
from CrySPY.IO import read_input as rin
from CrySPY.start import cryspy_init, cryspy_restart

from aiida_lammps.tests.utils import (
    get_or_create_local_computer, get_or_create_code)
from aiida_lammps.tests.utils import lammps_version

from aiida.engine import run_get_node, run, submit
from aiida.plugins import DataFactory, WorkflowFactory
import numpy as np
from aiida.orm import Code
from aiida.orm import Str, Dict, Int, List, Float
from aiida.engine import calcfunction, WorkChain, append_
from aiida.plugins import DataFactory
from itertools import cycle
import os
import io
from configparser import ConfigParser

from CrySPY.gen_struc.random.gen_pyxtal import Rnd_struc_gen_pyxtal


In [ ]:
import aiida
aiida.load_profile()


In [ ]:
SIMULATOR_PREFIX = 'simulator_'
ID_PREFIX = 'ID_'


In [ ]:
# nodebank settings

from tools.nodebank import NodeBank
pwd = os.getcwd()
print(pwd)
nodebank = NodeBank(pwd)


In [ ]:
!pwd

In [ ]:
import os
import numpy as np


from aiida.plugins import DataFactory
from aiida.orm import Code
from aiida.orm import Str, Dict, List, Int
from aiida.engine import calcfunction, WorkChain

from CrySPY.IO import read_input as rin
from pymatgen.io.vasp.inputs import Poscar
from pymatgen.core import Structure

# load types
StructureData = DataFactory('structure')
FolderData = DataFactory('folder')
SinglefileData = DataFactory('singlefile')
ArrayData = DataFactory('array')
LammpsPotential = DataFactory('lammps.potential')
TrajectoryData = DataFactory('array.trajectory')

# PandasFrameData = DataFactory('cryspy.dataframe')
PandasFrameData = DataFactory('dataframe.frame')

ConfigparserData = DataFactory('cryspy.configparser')
StructurecollectionData = DataFactory('cryspy.structurecollection')
EAData = DataFactory('cryspy.ea_data')
EAidData = DataFactory('cryspy.ea_id_data')


In [ ]:
pwd = os.getcwd()
cryspy_in = os.path.join(pwd, 'cryspy.in0')
inputs = {'cryspy_in': Str(cryspy_in)}
initialize_WorkChain = WorkflowFactory('cryspy.initial_structures')
initial_result = run(initialize_WorkChain, **inputs)


In [ ]:
# workflow結果表示
initial_result


In [ ]:
stat = initial_result["stat"]


In [ ]:
rslt_data_node = initial_result["rslt_data"]
rslt_data_node.df


In [ ]:
import shutil
# cryspy.inは実行中に書き換えられる。
shutil.copy("cryspy.in0", "cryspy.in")


# 

In [ ]:
# all the initial structures

all_initial_structures_node = initial_result["init_struc"]


# select structures to optimize

In [ ]:
        


class select_structure_to_run_WorkChain(WorkChain):
    @classmethod
    def define(cls, spec):
        super().define(spec)
        spec.input("ea_id_data", valid_type=EAidData, help='ea_id_data.')
        spec.input("init_struc", valid_type=StructurecollectionData, help='initial structures.')
        spec.outline(
            cls.select_structures
        )

        spec.output("init_struc", valid_type=StructurecollectionData, help='selected initial structures.')
        spec.output("work_path", valid_type=Dict, help='directory to saved results.')
        spec.output("ea_id_data", valid_type=EAidData)
        
    def select_structures(self):
        ea_id_node = self.inputs.ea_id_data
        structure_node = self.inputs.init_struc
        
        ea_id_data = ea_id_node.ea_id_data
        structures = structure_node.structurecollection
        id_queueing = ea_id_data[1]
        work_path_dic = {}
        structures_dic = {}
        for cid in id_queueing:
            work_path = './work/{:06}/'.format(cid)
            work_path = os.path.abspath(work_path)
            structures_dic[cid] = structures[cid]
            ID = str(cid)
            work_path_dic[ID] = work_path
        
        struc = StructurecollectionData(structures_dic)
        struc.store()
        self.out('init_struc', struc)
        
        d = Dict(dict=work_path_dic)
        d.store()
        self.out('work_path', d)
        
        id_queueing  = []
        ea_id = EAidData((ea_id_data[0],id_queueing, ea_id_data[2]))
        ea_id.store()
        self.out('ea_id_data', ea_id)
    

In [ ]:
inputs = {"ea_id_data":  initial_result["ea_id_data"], "init_struc":initial_result["init_struc"] }

select_structure_result = run(select_structure_to_run_WorkChain, **inputs)
work_path_node = select_structure_result["work_path"]
structures_to_optimize_node = select_structure_result["init_struc"]


In [ ]:
select_structure_result

In [ ]:
ea_id_node = select_structure_result["ea_id_data"]
ea_id_node.ea_id_data

In [ ]:
computer_local = 'localhost'
code_lammps_opt = get_or_create_code('lammps.optimize',
                                     computer_local, 'lammps')
meta_options = {
    "resources": {
        "num_machines": 1,
        "num_mpiprocs_per_machine": 4}
}
code_lammps_opt.label


In [ ]:
# lammps potentials
pair_style = 'tersoff'
potential_dict = {
    'Ga Ga Ga': '1.0 0.007874 1.846 1.918000 0.75000 -0.301300 1.0 1.0 1.44970 410.132 2.87 0.15 1.60916 535.199',
    'N  N  N': '1.0 0.766120 0.000 0.178493 0.20172 -0.045238 1.0 1.0 2.38426 423.769 2.20 0.20 3.55779 1044.77',
    'Ga Ga N': '1.0 0.001632 0.000 65.20700 2.82100 -0.518000 1.0 0.0 0.00000 0.00000 2.90 0.20 0.00000 0.00000',
    'Ga N  N': '1.0 0.001632 0.000 65.20700 2.82100 -0.518000 1.0 1.0 2.63906 3864.27 2.90 0.20 2.93516 6136.44',
    'N  Ga Ga': '1.0 0.001632 0.000 65.20700 2.82100 -0.518000 1.0 1.0 2.63906 3864.27 2.90 0.20 2.93516 6136.44',
    'N  Ga N ': '1.0 0.766120 0.000 0.178493 0.20172 -0.045238 1.0 0.0 0.00000 0.00000 2.20 0.20 0.00000 0.00000',
    'N  N  Ga': '1.0 0.001632 0.000 65.20700 2.82100 -0.518000 1.0 0.0 0.00000 0.00000 2.90 0.20 0.00000 0.00000',
    'Ga N  Ga': '1.0 0.007874 1.846 1.918000 0.75000 -0.301300 1.0 0.0 0.00000 0.00000 2.87 0.15 0.00000 0.00000'}
potential = DataFactory("lammps.potential")(
    type=pair_style, data=potential_dict
)
potential.attributes


In [ ]:
# lammps.optimize parameters
parameters = DataFactory('dict')(dict={
    'lammps_version': lammps_version(),
    'output_variables': ["temp", "etotal", "pe", "ke"],
    'thermo_keywords': [],
    'units': 'metal',
    'relax': {
        'type': 'aniso',
        'pressure': 0.0,
        'vmax': 0.005,
    },
    "minimize": {
        'style': 'cg',
        'energy_tolerance': 1.0e-5,
        'force_tolerance': 1.0e-3,
        'max_iterations': 1000,
        'max_evaluations': 10000
        }
})


In [ ]:
code_lammps_opt


In [ ]:
if len(all_initial_structures_node.structurecollection.keys())>0 and len(work_path_node.get_dict())>0:
    inputs = {'code_string': Str(code_lammps_opt.label),
             "initial_structures": structures_to_optimize_node, 
             "cwd": work_path_node,
             "potential": potential,
             "parameters": parameters}
    optimization_simulator_lammps_WorkChain = WorkflowFactory('cryspy.optimize_structures_lammps')
    optimize_result, node = run_get_node(optimization_simulator_lammps_WorkChain, **inputs)
    # finish_job(jobs)
    

In [ ]:
print(optimize_result["results"].get_dict())


In [ ]:
all_optimized_structures_node = optimize_result["final_structures"]
len(all_optimized_structures_node.structurecollection.keys())


# next run

In [ ]:
# 修正すること。
# stat, struc_dict = cryspy_restart.restart(all_initial_structures_node.structurecollection)
ea_id_node = initial_result["ea_id_data"]

In [ ]:
# optの対称性の登録
@calcfunction
def generate_rlst(all_initial_structures_node: StructurecollectionData, 
                  all_optimized_structures_node: StructurecollectionData, 
                  optimize_result: Dict, 
                  ea_id_data_node: EAidData, 
                  rslt_data_node: PandasFrameData):
    init_struc_data = all_initial_structures_node.structurecollection
    opt_struc = all_optimized_structures_node.structurecollection
    opt_results = optimize_result.get_dict()
    rslt_data = rslt_data_node.df
    gen = ea_id_data_node.ea_id_data[0]
    magmon = None
    check_opt = 'done'
    
    symprec = rin.symprec
    for i,energy in zip(opt_results["index"], opt_results["energy"]):
        if energy is not None:
            check_opt = 'done'
        else:
            check_opt = 'not_yet'
        if energy is None:
            energy = np.nan
        else:
            natot = len(init_struc_data[i].atomic_numbers)
            energy /= natot
            
        in_spg = init_struc_data[i].get_space_group_info(symprec=symprec)
        opt_spg = opt_struc[i].get_space_group_info(symprec=symprec)
        rslt_data.loc[i] = [gen, in_spg[1], in_spg[0], opt_spg[1], opt_spg[0], 
                           energy, magmon, check_opt]
        
    return PandasFrameData(rslt_data)
    # return rslt_data

rslt_data_node_new = generate_rlst(all_initial_structures_node, all_optimized_structures_node, 
                              optimize_result["results"], 
                              ea_id_node, rslt_data_node)

# rslt for new EA
rslt_data_node_new.df

# next_sg

It generates new candidates by EA.


In [ ]:
ea_data_node = initial_result["ea_data"]
rslt_data_node = rslt_data_node_new

inputs = {'initial_structures': all_initial_structures_node,
         'optimized_structures': all_optimized_structures_node,
          'rslt_data': rslt_data_node,
          'ea_id_data': ea_id_node,
          'ea_data': ea_data_node,
         'stat': stat}
next_sg_WorkChain = WorkflowFactory('cryspy.next_sg')
next_sg_results = run(next_sg_WorkChain, **inputs)


In [ ]:
next_sg_results

In [ ]:
stat = next_sg_results["stat"]
stat.get_dict()

In [ ]:
ea_id_node = next_sg_results["ea_id_data"]
print(ea_id_node.ea_id_data)


In [ ]:
all_initial_structures_node = next_sg_results["initial_structures"]
all_initial_structures_node.structurecollection.keys()

In [ ]:
next_sg_results["rslt_data"].df.sort_values(by="E_eV_atom")

In [ ]:
all_initial_structures_node.structurecollection.keys()

# next structure  optimization

In [ ]:
inputs = {"ea_id_data":  ea_id_node, "init_struc": all_initial_structures_node }

select_structure_result = run(select_structure_to_run_WorkChain, **inputs)
work_path_node = select_structure_result["work_path"]
structures_to_optimize_node = select_structure_result["init_struc"]
print(work_path_node.get_dict().keys(),  structures_to_optimize_node.get_dict().keys())

In [ ]:
if len(structures_to_optimize_node.structurecollection.keys())>0 and len(work_path_node.get_dict())>0:
    inputs = {'code_string': Str(code_lammps_opt.label),
             "initial_structures": structures_to_optimize_node, 
             "cwd": work_path_node,
             "potential": potential,
             "parameters": parameters}
    optimization_simulator_lammps_WorkChain = WorkflowFactory('cryspy.optimize_structures_lammps')
    optimize_result, node = run_get_node(optimization_simulator_lammps_WorkChain, **inputs)
    # finish_job(jobs)

In [ ]:
print(optimize_result["results"].get_dict())

In [ ]:
# merge all_optimized_structures_node
all_optimized_structures_node.structurecollection.keys(), optimize_result["final_structures"].structurecollection.keys()

In [ ]:
import copy
@calcfunction
def merge_structurecollection(struc1: StructurecollectionData, struc2: StructurecollectionData):
    struc1 = struc1.structurecollection
    struc2 = struc2.structurecollection
    for key, value in struc2.items():
        if key not in struc1:
            struc1[key] = value
        else:
            print(f"Warning: key {key} is in struc1.")
    return StructurecollectionData(struc1)

In [ ]:
all_optimized_structures_node = merge_structurecollection(all_optimized_structures_node, optimize_result["final_structures"])
all_optimized_structures_node.structurecollection.keys()

In [ ]:
ea_id_node = next_sg_results["ea_id_data"]
all_initial_structures_node = next_sg_results["initial_structures"]
all_initial_structures_node.structurecollection.keys(), all_optimized_structures_node.structurecollection.keys()

In [ ]:
rslt_data_node = generate_rlst(all_initial_structures_node, all_optimized_structures_node, 
                              optimize_result["results"], 
                              ea_id_node, rslt_data_node)
rslt_data_node.df.sort_values(by="E_eV_atom")

In [ ]:
ea_data_node = next_sg_results["ea_data"]
ea_id_data_node = select_structure_result["ea_id_data"]


In [ ]:
print(ea_id_data_node.get_dict())


In [ ]:
all_optimized_structures_node.structurecollection.keys()

# next_sg

In [ ]:
rslt_data_node.df.sort_values(by='E_eV_atom')

In [ ]:
stat["basic"]

In [ ]:
inputs = {'initial_structures': all_initial_structures_node,
         'optimized_structures': all_optimized_structures_node,
          'rslt_data': rslt_data_node,
          'ea_id_data': ea_id_data_node,
          'ea_data': ea_data_node,
         'stat': stat}

next_sg_WorkChain = WorkflowFactory('cryspy.next_sg')
next_sg_results = run(next_sg_WorkChain, **inputs)


In [ ]:
next_sg_results

In [ ]:
ea_id_data = next_sg_results['ea_id_data']


In [ ]:
all_initial_structures_node = next_sg_results["initial_structures"]
print(all_initial_structures_node.get_dict().keys())


# structure optimization 

In [ ]:
inputs = {"ea_id_data":  ea_id_data, "init_struc": all_initial_structures_node }

select_structure_result = run(select_structure_to_run_WorkChain, **inputs)
work_path_node = select_structure_result["work_path"]
structures_to_optimize_node = select_structure_result["init_struc"]


In [ ]:
work_path_node.get_dict(), structures_to_optimize_node.get_dict().keys()


In [ ]:
if len(structures_to_optimize_node.structurecollection.keys())>0 and len(work_path_node.get_dict())>0:
    inputs = {'code_string': Str(code_lammps_opt.label),
             "initial_structures": structures_to_optimize_node, 
             "cwd": work_path_node,
             "potential": potential,
             "parameters": parameters}
    optimization_simulator_lammps_WorkChain = WorkflowFactory('cryspy.optimize_structures_lammps')
    optimize_result, node = run_get_node(optimization_simulator_lammps_WorkChain, **inputs)
    # finish_job(jobs)
    

In [ ]:
optimize_result

In [ ]:
all_optimized_structures_node = merge_structurecollection(all_optimized_structures_node, optimize_result["final_structures"])
all_optimized_structures_node.structurecollection.keys()


In [ ]:
ea_id_node = next_sg_results["ea_id_data"]
all_initial_structures_node = next_sg_results["initial_structures"]

all_initial_structures_node.structurecollection.keys(), all_optimized_structures_node.structurecollection.keys()


In [ ]:
rslt_data_node_new = generate_rlst(all_initial_structures_node, all_optimized_structures_node, 
                              optimize_result["results"], 
                              ea_id_node, rslt_data_node)
print(rslt_data_node_new.df.shape)
rslt_data_node_new.df.sort_values(by="E_eV_atom").head(50)